In [4]:
import cv2
import numpy as np
import os
import pandas as pd
from datetime import datetime
import openpyxl
from openpyxl.styles import PatternFill

# HSV color ranges
COLOR_RANGES = {
    'Red': ([0, 100, 100], [10, 255, 255]),
    'Green': ([40, 50, 50], [80, 255, 255]),
    'Blue': ([100, 150, 0], [140, 255, 255])
}

# Counter for statistics
color_counts = {
    'Red': 0,
    'Green': 0,
    'Blue': 0,
    'Other': 0
}

# Excel filename
excel_filename = 'color_detection_results.xlsx'

def detect_color(hsv, image, image_name=""):
    mask_combined = None
    detected_colors = []

    for color_name, (lower, upper) in COLOR_RANGES.items():
        lower_np = np.array(lower)
        upper_np = np.array(upper)
        mask = cv2.inRange(hsv, lower_np, upper_np)

        mask = cv2.GaussianBlur(mask, (5, 5), 0)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((5, 5), np.uint8))

        if np.any(mask):
            color_counts[color_name] += 1
            detected_colors.append((color_name, mask))

    if not detected_colors:
        color_counts['Other'] += 1
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        _, other_mask = cv2.threshold(gray, 10, 255, cv2.THRESH_BINARY)
        detected_colors.append(('Other', other_mask))

    for color_name, mask in detected_colors:
        result = cv2.bitwise_and(image, image, mask=mask)
        hex_color = get_hex_color_from_mask(result)

        if hex_color:
            hex_color_clean = hex_color.lstrip('#')
            bgr = tuple(int(hex_color_clean[i:i+2], 16) for i in (4, 2, 0))
            cv2.rectangle(image, (10, 10), (110, 60), bgr, -1)
            text = f'{color_name} ({hex_color})'
            cv2.putText(image, text, (120, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            print(f"{image_name}  ->  {color_name}: {hex_color}")
            return color_name, hex_color
    return None, None

def get_hex_color_from_mask(result):
    mean_val = cv2.mean(result)
    bgr = mean_val[:3]

    if all(v == 0 for v in bgr):
        return None

    hex_color = '#{:02x}{:02x}{:02x}'.format(int(bgr[2]), int(bgr[1]), int(bgr[0]))
    if len(hex_color) != 7:
        return None
    return hex_color
def save_to_excel(data):
    df = pd.DataFrame(data, columns=["Image Name", "Color Name", "Color Code", "Color"])

    if os.path.exists("folder_results.xlsx"):
        existing_df = pd.read_excel("folder_results.xlsx")
        df = pd.concat([existing_df, df], ignore_index=True)

    df.to_excel("folder_results.xlsx", index=False)

    wb = openpyxl.load_workbook("folder_results.xlsx")
    sheet = wb.active

    for row in range(2, sheet.max_row + 1):  # ✅ fix: color all rows, not just new ones
        color_code = sheet.cell(row=row, column=3).value
        if color_code:
            try:
                fill = PatternFill(start_color=color_code.lstrip('#'),
                                   end_color=color_code.lstrip('#'),
                                   fill_type="solid")
                cell = sheet.cell(row=row, column=4)
                cell.value = ""
                cell.fill = fill
            except ValueError:
                pass

    wb.save("folder_results.xlsx")

def camera_mode():
    cap = cv2.VideoCapture(0)
    data = []

    print("Press 'q' to exit.")
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        color_name, color_code = detect_color(hsv, frame)

        if color_name and color_code:
            now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            data.append([now, color_name, color_code, ""])

        cv2.imshow("Camera", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    save_to_excel(data)

def folder_mode():
    folder_path = "img_test"
    data = []

    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(folder_path, filename)
            image = cv2.imread(image_path)
            if image is None:
                continue
            hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
            color_name, color_code = detect_color(hsv, image, image_name=filename)

            if color_name and color_code:
                data.append([filename, color_name, color_code, ""])  # Leave Color column empty

            cv2.imshow("Image", image)
            cv2.waitKey(2000)
            cv2.destroyAllWindows()

    save_to_excel(data)

def print_statistics():
    print("\nColor Statistics:")
    for color, count in color_counts.items():
        print(f"{color}: {count}")

def main(choice):
    if choice == "1":
        camera_mode()
    elif choice == "2":
        folder_mode()
    else:
        print("Invalid choice. Please enter 1 or 2.")
        return
    print_statistics()

if __name__ == "__main__":
    print("Choose mode:")
    print("1: From Camera")
    print("2: From Folder")
    user_choice = input("Enter number (1 or 2): ").strip()

    while user_choice not in ["1", "2"]:
        user_choice = input("Invalid choice. Please enter 1 or 2: ").strip()

    main(user_choice)


Choose mode:
1: From Camera
2: From Folder


img1.jpg  ->  Red: #fe0000
img10.jpg  ->  Green: #01ca20
img2.jpg  ->  Other: #edd400
img3.jpg  ->  Blue: #4d83bf
img4.jpg  ->  Other: #00a3e8
img5.jpg  ->  Blue: #0000fe
img6.jpg  ->  Other: #ffff00
img7.jpg  ->  Blue: #000000
img8.jpg  ->  Other: #fcfcfc
img9.jpg  ->  Green: #65ff00

Color Statistics:
Red: 1
Green: 2
Blue: 3
Other: 4
